# Rich Logging and Printing

In [1]:
# default_exp logger

In [2]:
# export
from rich.console import Console
from rich.theme import Theme
console = Console(theme=Theme({
    "repr.number": "bold cyan",
    "repr.string": "bold green",
    "logging.level.info": "dim yellow",
    "logging.level.warning": "dim red",
    "logging.level.exception": "bold red"
}))

print = console.print

In [3]:
print('The Number is 128')
print({'a':1, 'b': [{1,2,3}, 'lskjdf', ]})

In [4]:
# export
from loguru import logger
from datetime import datetime
from fastcore.basics import patch_to
from rich.logging import RichHandler
from pathlib import Path

@patch_to(RichHandler)
def render(
    self,
    *,
    record,
    traceback,
    message_renderable: "ConsoleRenderable",
) -> "ConsoleRenderable":
    """patched the renderer to print function name as well
    """
    path = Path(record.pathname).name
    level = self.get_level_text(record)
    time_format = None if self.formatter is None else self.formatter.datefmt
    log_time = datetime.fromtimestamp(record.created)

    log_renderable = self._log_render(
        self.console,
        [message_renderable] if not traceback else [message_renderable, traceback],
        log_time=log_time,
        time_format=time_format,
        level=level,
        path=path,
        line_no=f'{record.funcName}:{record.lineno}',
        link_path=record.pathname if self.enable_link_path else None,
    )
    return log_renderable

logger.configure(handlers=[{
    "sink":RichHandler(rich_tracebacks=True, console=console),
    "format":'<level>{message}</level>'}
])

logger = logger

Info  = lambda x: logger.info(x)
Warn  = lambda x: logger.warning(x)
Debug = lambda x: logger.debug(x)
Excep = lambda x: logger.exception(x)

In [5]:
logger.debug('TESTING {1,2,3}')
logger.info('TESTING {1,2,3}')
logger.warning('TESTING {1,2,3}')
logger.exception('TESTING {1,2,3}')
